In [1]:
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# %pip install openpyxl

In [39]:
df = pd.read_excel('../data/guokao2024.xlsx')
df.shape

(115640, 7)

In [40]:
df

,准考证号,招录机关,部门代码,用人司局,招考职位,职位代码,最低面试分数
0,2111021301304,中央办公厅,2000,警卫局,档案管理岗位一级主任科员及以下,100110001001,110.1
1,2111071000324,中央办公厅,2000,警卫局,档案管理岗位一级主任科员及以下,100110001001,110.1
2,2111071401708,中央办公厅,2000,警卫局,档案管理岗位一级主任科员及以下,100110001001,110.1
3,2142050200204,中央办公厅,2000,警卫局,档案管理岗位一级主任科员及以下,100110001001,110.1
4,2161010109521,中央办公厅,2000,警卫局,档案管理岗位一级主任科员及以下,100110001001,110.1
...,...,...,...,...,...,...,...
115635,191111011900629,国家数据局,191000,机关司局五,机关司局一级主任科员及以下,200110005002,106.4
115636,191111120500430,国家数据局,191000,机关司局五,机关司局一级主任科员及以下,200110005002,106.4
115637,191111120602408,国家数据局,191000,机关司局五,机关司局一级主任科员及以下,200110005002,106.4
115638,191137090100804,国家数据局,191000,机关司局五,机关司局一级主任科员及以下,200110005002,106.4


In [44]:
df['招录机关-用人司局-招考职位'] = df.apply(lambda row: f"{row['招录机关']}_{row['用人司局']}_{row['招考职位']}", axis=1)

In [47]:
print("职位代码去重后总数: " + str(df['职位代码'].nunique()))
print("招录机关-用人司局-招考职位组合去重后总数: " + str(df['招录机关-用人司局-招考职位'].nunique()))

职位代码去重后总数: 6493
招录机关-用人司局-招考职位组合去重后总数: 16006


In [48]:
df['最低面试分数'].describe()

count    115640.000000
mean        112.478784
std          23.569302
min          45.250000
25%         111.800000
50%         120.600000
75%         126.600000
max         144.700000
Name: 最低面试分数, dtype: float64

## 进行分类，考专业课和不考专业课

In [49]:
df_below_89 = df[df['最低面试分数'] < 89]
df_above_89 = df[df['最低面试分数'] >= 89]

In [55]:
print("包含专业课的岗位总数: " + str(df_below_89['招录机关-用人司局-招考职位'].nunique()))
print("不含专业课的岗位总数: " + str(df_above_89['招录机关-用人司局-招考职位'].nunique()))

包含专业课的岗位总数: 1760
不含专业课的岗位总数: 14246


In [50]:
df_below_89['最低面试分数'].describe()

count    16598.000000
mean        58.278703
std          4.996526
min         45.250000
25%         54.831250
50%         58.472500
75%         61.510000
max         72.925000
Name: 最低面试分数, dtype: float64

In [51]:
df_above_89['最低面试分数'].describe()

count    99042.00000
mean       121.56193
std          8.34317
min         90.00000
25%        116.60000
50%        122.40000
75%        127.50000
max        144.70000
Name: 最低面试分数, dtype: float64

In [34]:
df[(df['最低面试分数'] >= 90) & (df['最低面试分数'] <= 91)]['招录机关'].drop_duplicates()

10628               长江海事局
11932               广西海事局
12817          水利部海河水利委员会
16802         中国人民银行安徽省分行
23648              呼和浩特海关
24194               哈尔滨海关
35326        国家税务总局河北省税务局
40095     国家税务总局内蒙古自治区税务局
47001       国家税务总局黑龙江省税务局
63678        国家税务总局江西省税务局
75983        国家税务总局湖南省税务局
102241        国家统计局江西调查总队
103290        国家统计局四川调查总队
104741           西藏自治区气象局
104803             甘肃省气象局
114426      国家矿山安全监察局内蒙古局
114450       国家矿山安全监察局吉林局
114484      国家矿山安全监察局黑龙江局
114787       国家矿山安全监察局甘肃局
Name: 招录机关, dtype: object